<a href="https://colab.research.google.com/github/Alighiamy/Numpy/blob/main/ABD_CLPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classical Laminated Plate Theory
## ABD Matrix of CLPT
_*__writer:__*_ 
* Ali Ghiamy

In [1]:
%reset-f
import pandas as pd
import numpy as np
π = np.pi

In [2]:
#---------------------------------------------------------
# Geometry of Plate Inputs
ht = 1 # Total Hight of Plate
b̄ = 50 # Length of Plate in "Y" Direction
ā = 50 # Length of Plate in "X" Direction

NL= 1 # Number of Layers
#---------------------------------------------------------
# Material Peroperties of Plate:
# Isotropic Plate
E1 = np.array([208000])
E2 = E1
ν12 = np.array([0.3])
G12 = np.array([E2[0]/(2*(1+ν12[0]))])
G23 = G12
G13 = G12

ρ=7800 # Kg/cm3
#---------------------------------------------------------
# Orientation of ech Ply:
θ = [0,90,90,0]

print(E1,E2,ν12,G12,G23,G13)

[208000] [208000] [0.3] [80000.] [80000.] [80000.]


In [3]:
# Laminate Parameters:
# Puasons Ratio
ν21 = []
for i in range(NL):
    ν21.append(E2[i]/E1[i]*ν12[i])

# Density of Each Ply
ρi = np.ones((NL,),dtype=int)*ρ

# Coordinates of each Face of Layers According to Thickness of Each Ply
Zt=[]
Zt.append(-ht/2)
for i in range(NL):
    Zt.append(Zt[i]+ht/NL)

print(Zt)

[-0.5, 0.5]


In [4]:
# Stiffnes Matrix of Material for Bending in Local Direction: "QB"
QB=np.zeros((NL,3,3),dtype=int)
for i in range(NL):
    QB[i][0][0] = E1[i]/(1-ν12[i]*ν21[i])
    QB[i][0][1] = E2[i]*ν12[i]/(1-ν12[i]*ν21[i])
    QB[i][1][0] = E2[i]*ν12[i]/(1-ν12[i]*ν21[i])
    QB[i][1][1] = E2[i]/(1-ν12[i]*ν21[i])
    QB[i][2][2] = G12[i]

print(QB[0][:][:])


[[228571  68571      0]
 [ 68571 228571      0]
 [     0      0  80000]]


In [5]:
# Rotation Matrix
Cθ = np.cos(np.radians(θ))
Sθ = np.sin(np.radians(θ))

MatrixA = np.zeros((NL,3,3),dtype=int)
for i in range(NL):
    MatrixA[:][:][i] = [[Cθ[i]**2   ,Sθ[i]**2    ,-2*Cθ[i]*Sθ[i]],
                        [Sθ[i]**2   ,Cθ[i]**2    ,2*Cθ[i]*Sθ[i]],
                        [Cθ[i]*Sθ[i],-Cθ[i]*Sθ[i],Cθ[i]**2-Sθ[i]**2]]

MatrixB = np.zeros((NL,3,3),dtype=int)
for i in range(NL):
    MatrixB[:][:][i] = [[Cθ[i]**2      ,Sθ[i]**2     ,Cθ[i]*Sθ[i]],
                        [Sθ[i]**2      ,Cθ[i]**2     ,-Cθ[i]*Sθ[i]],
                        [-2*Cθ[i]*Sθ[i],2*Cθ[i]*Sθ[i],Cθ[i]**2-Sθ[i]**2]]

# Stiffnes Matrix of Material for Bending in Local Direction: "QB"
Q̄B = np.zeros((NL,3,3))
for i in range(NL):
    Q̄B[i][:][:] = MatrixA[i][:][:]@QB[i][:][:]@MatrixB[i][:][:]

print(Q̄B[0][:][:])

[[228571.  68571.      0.]
 [ 68571. 228571.      0.]
 [     0.      0.  80000.]]


In [6]:
# ABD Matrix for CLPT
AA = np.zeros((3,3))
BB = np.zeros((3,3))
DD = np.zeros((3,3))

for i in range(NL):
    AA[:][:] += Q̄B[i][:][:]*(Zt[i+1]-Zt[i])
    BB[:][:] += (1/2)*Q̄B[i][:][:]*(Zt[i+1]**2-Zt[i]**2)
    DD[:][:] += (1/3)*Q̄B[i][:][:]*(Zt[i+1]**3-Zt[i]**3)

print('A=',AA)
print("*```````````````*")
print('D=',DD)

A= [[228571.  68571.      0.]
 [ 68571. 228571.      0.]
 [     0.      0.  80000.]]
*```````````````*
D= [[19047.58333333  5714.25           0.        ]
 [ 5714.25       19047.58333333     0.        ]
 [    0.             0.          6666.66666667]]


In [10]:
##%
Dbend = np.zeros((6,6),dtype=int)
'''
Dbend=[A B
       B D]
'''
#----A[1,1]:
# A[1,1:3]
Dbend[0][0] = AA[0][0]
Dbend[0][1] = AA[0][1]
Dbend[0][2] = AA[0][2]
# A[2,1:3]
Dbend[1][0] = AA[1][0]
Dbend[1][1] = AA[1][1]
Dbend[1][2] = AA[1][2]
# A[3,1:3]
Dbend[2][0] = AA[2][0]
Dbend[2][1] = AA[2][1]
Dbend[2][2] = AA[2][2]

#----B[1,2]:-------------
# B[1,1:3]
Dbend[0][3] = BB[0][0]
Dbend[0][4] = BB[0][1]
Dbend[0][5] = BB[0][2]
# B[2,1:3]
Dbend[1][3] = BB[1][0]
Dbend[1][4] = BB[1][1]
Dbend[1][5] = BB[1][2]
# B[3,1:3]
Dbend[2][3] = BB[2][0]
Dbend[2][4] = BB[2][1]
Dbend[2][5] = BB[2][2]

#----B[2,1]:-------------
# B[1,1:3]
Dbend[3][0] = BB[0][0]
Dbend[3][1] = BB[0][1]
Dbend[3][2] = BB[0][2]
# B[2,1:3]
Dbend[4][0] = BB[1][0]
Dbend[4][1] = BB[1][1]
Dbend[4][2] = BB[1][2]
# B[3,1:3]
Dbend[5][0] = BB[2][0]
Dbend[5][1] = BB[2][1]
Dbend[5][2] = BB[2][2]

#----D[2,2]:-------------
# D[1,1:3]
Dbend[3][3] = DD[0][0]
Dbend[3][4] = DD[0][1]
Dbend[3][5] = DD[0][2]
# D[2,1:3]
Dbend[4][3] = DD[1][0]
Dbend[4][4] = DD[1][1]
Dbend[4][5] = DD[1][2]
# D[3,1:3]
Dbend[5][3] = DD[2][0]
Dbend[5][4] = DD[2][1]
Dbend[5][5] = DD[2][2]

print("*```````````````*")
Dbend

*```````````````*


array([[228571,  68571,      0,      0,      0,      0],
       [ 68571, 228571,      0,      0,      0,      0],
       [     0,      0,  80000,      0,      0,      0],
       [     0,      0,      0,  19047,   5714,      0],
       [     0,      0,      0,   5714,  19047,      0],
       [     0,      0,      0,      0,      0,   6666]])